## Single run example of nanotribological properties workflow

### Background
- Using SAM as coating to reduce coefficient of friction and force of adhesion when shearing between two surface
- Many variables that can be varied and, hence, makes it a prime candidate for Computational Simulation
- Several MD studies focusing on different properties
    - Andrew Paper: backbone chain length, chain density, terminal groups
    - Current screening project: mixed composition 
    - Near future project: backbone chemitries
- The legacy encourage this research setup to be TRUE
    - Future research can continue this project by varing and/or adding more variables based on the original project
    
### Model
- Initialization of two surfaces coated with SAM - varying mixing composition, backbone chainlength, terminal groups
- Two surface was then energy minimized through a few steps (by LAMMPS and GROMACS)
- The two system is then compressed and sheared against each other (the top surface moving, and the bottom surface fixed)
- The output is analyzed with mdanalysis

### Simulation Details
- Initialze by MoSDeF
- Fix overlaps by LAMMPS
- Energy Minimize by GROMACS
- NVT Equilibrate by GROMACS
- Compress by GROMACS
- Shearing at 5nN, 15nN, 25nN by GROMACS
- Data Analysis with mdanalysis

#### System initialization

In [5]:
import flow
import numpy as np
import scipy
from foyer import Forcefield
from mbuild.formats.lammpsdata import write_lammpsdata
from mbuild.lib.atoms import H
from ctools.fileio import write_monolayer_ndx, read_ndx
from ctools.lib.chains import Alkylsilane
from ctools.lib.recipes import DualSurface, SilicaInterface, SurfaceMonolayer
from util.index_groups import generate_index_groups

In [10]:
""" Define system variable"""
chainlength = 17
backbone_A = Alkylsilane
backbone_B = Alkylsilane
seed = 27
pattern_type = "random"
terminal_group = "methyl"
num_chains = 100
"""
-----------------------------------
Generate amorphous silica interface
-----------------------------------
"""
surface_a = SilicaInterface(thickness=1.2, seed=seed)
surface_b = SilicaInterface(thickness=1.2, seed=seed)

"""
------------------------------------------------------
Generate prototype of functionalized alkylsilane chain
------------------------------------------------------
"""
chain_prototype_A = backbone_A(
    chain_length=chainlength, terminal_group=terminal_group)
chain_prototype_B = backbone_B(
    chain_length=chainlength, terminal_group=terminal_group)
"""
----------------------------------------------------------
Create monolayer on surface, backfilled with hydrogen caps
----------------------------------------------------------
"""
# bottom monolayer is backfilled with the other terminal group
# num_chains = num_chains * a_fraction
monolayer_a = SurfaceMonolayer(
    surface=surface_a,
    chains=chain_prototype_A,
    n_chains=num_chains,
    seed=seed,
    backfill=H(),
    rotate=False,
)
monolayer_a.name = "Bottom"
monolayer_b = SurfaceMonolayer(
    surface=surface_b,
    chains=chain_prototype_B,
    n_chains=num_chains,
    seed=seed,
    backfill=H(),
    rotate=False,
)
monolayer_b.name = "Top"

"""
----------------------
Create dual monolayers
----------------------
"""
dual_monolayer = DualSurface(
    bottom=monolayer_a, top=monolayer_b, separation=2.0
)

"""
--------------------------------------------------------
Make sure box is elongated in z to be pseudo-2D periodic
--------------------------------------------------------
"""
box = dual_monolayer.boundingbox
dual_monolayer.periodicity += np.array([0, 0, 5.0 * box.lengths[2]])

"""
-------------------------------------------------------------------
- Save to .GRO, .TOP, and .LAMMPS formats
- Atom-type the system using Foyer, with parameters from the OPLS
force field obtained from GROMACS. Parameters are located in a
Foyer XML file in the `atools` git repo, with references provided
as well as notes where parameters have been added or altered to
reflect the literature.
-------------------------------------------------------------------
"""

forcefield_filepath = "./util/forcefield/oplsaa.xml"

dual_monolayer.save("init.gro", residues=["Top", "Bottom"], overwrite=True)

structure = dual_monolayer.to_parmed(
    box=None, residues=["Top", "Bottom"]
    )
ff = Forcefield(forcefield_files=forcefield_filepath)
structure = ff.apply(structure)
structure.combining_rule = "geometric"

structure.save("init.top", overwrite=True)
write_lammpsdata(filename="init.lammps", structure=structure)

"""
--------------------------------------
Specify index groups and write to file
--------------------------------------
"""
index_groups = generate_index_groups(
    system=dual_monolayer,
    terminal_group=terminal_group,
    freeze_thickness=0.5,
)
write_monolayer_ndx(rigid_groups=index_groups, filename="init.ndx")

/Users/daico007/Documents/Code/Simulation/mosdef36/ctools/ctools/lib/recipes/monolayer.py:85: UserWarning: 
 Adding 100 of chain <Alkylsilane 60 particles, non-periodic, 59 bonds, id: 5038421552>
  warn("\n Adding {} of chain {}".format(len(pattern), chains[-1]))
/Users/daico007/Documents/Code/Simulation/mosdef36/ctools/ctools/lib/recipes/monolayer.py:85: UserWarning: 
 Adding 100 of chain <Alkylsilane 60 particles, non-periodic, 59 bonds, id: 5038421440>
  warn("\n Adding {} of chain {}".format(len(pattern), chains[-1]))
/Users/daico007/Documents/Code/Simulation/mosdef36/mbuild/mbuild/compound.py:2412: UserWarning: Guessing that "<OS pos=( 4.6333, 2.1916, 1.4235), 0 bonds, id: 5079010944>" is element: "O"
  atom, element))
/Users/daico007/Documents/Code/Simulation/mosdef36/foyer/foyer/forcefield.py:248: UserWarning: Parameters have not been assigned to all impropers. Total system impropers: 20204, Parameterized impropers: 0. Note that if your system contains torsions of Ryckaert-Belle

No urey bradley terms detected, will use angle_style harmonic
RB Torsions detected, will use dihedral_style opls
bottom_frozen: 636
top_frozen: 636
bottom_surface: 2132
top_surface: 2132
surfaces: 4264
bottom_chains: 6000
top_chains: 6000
chains: 12000
bottom_termini: 400
top_termini: 400
terminal_groups: 800
bottom: 8132
top: 8132
System: 16264


In [11]:
dual_monolayer.visualize()

/Users/daico007/Documents/Code/Simulation/mosdef36/mbuild/mbuild/compound.py:2412: UserWarning: Guessing that "<OS pos=( 4.6333, 2.1916, 1.4235), 0 bonds, id: 5114923720>" is element: "O"
  atom, element))


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

#### Fix overlaps by LAMMPS

In [ ]:
!{} -- lmp_titan -in {}/in.minimize -log {}/minimize.log

#### Energy minimize by GROMACS

In [ ]:
# Include conversion steps and grompp 

# Run GROMACS

#### NVT Equilibrate by GROMACS

In [ ]:
# Include conversion steps and grompp 

# Run GROMACS

#### Compress wih GROMACS

In [1]:
# Include conversion steps and grompp 

# Run GROMACS

#### Shearing at 5nN

In [ ]:
# Include conversion steps and grompp 

# Run GROMACS

#### Shearing at 15nN

In [ ]:
# Include conversion steps and grompp 

# Run GROMACS

#### Shearing at 25nN

In [ ]:
# Include conversion steps and grompp 

# Run GROMACS

#### Calculating COF and F0

In [ ]:
# Unwrap stuff

# Run COF and F0 calculation